In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml

class Perceptron:
    def __init__(self, n_inputs, learning_rate=0.01, n_iters=100, random_state=None):
        self.n_inputs = n_inputs
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        self.random_state = random_state
        self.n_outputs = 1

    def init_weights(self):
        rng = np.random.RandomState(self.random_state) if self.random_state else np.random
        self.weights = rng.uniform(low=-0.1, high=0.1, size=(self.n_inputs, self.n_outputs))
        self.bias = rng.uniform(low=-0.1, high=0.1, size=(self.n_outputs,))
    def activation_function(self, z):
        return 1 / (1 + np.exp(-z))

    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        a = self.activation_function(z)
        return np.where(a >= 0.5, 1, 0).flatten()  # Порог 0.5, flatten для совместимости с accuracy_score

    def train(self, X, y):
        n_samples, n_features = X.shape
        self.n_inputs = n_features

        # Инициализация весов и смещений
        self.init_weights()

        # Инициализация общей ошибки и коррекций весов
        E_total = 0
        delta_w = np.zeros_like(self.weights)
        delta_b = np.zeros_like(self.bias)

        # Критерий останова (максимальное количество эпох)
        for epoch in range(self.n_iters):
            E_total = 0  # Обнуляем общую ошибку для каждой эпохи
            delta_w = np.zeros_like(self.weights) # Обнуляем накопленные изменения весов
            delta_b = np.zeros_like(self.bias) # Обнуляем накопленные изменения смещений

            # Обработка каждого обучающего вектора
            for i in range(n_samples):
                X_m = X[i]
                d_m = y[i]  # Желаемый выход (целевая переменная)

                # Вычисление net_j и y_j (выхода)
                net_j = np.dot(X_m, self.weights) + self.bias
                y_j = self.activation_function(net_j)

                # Вычисление ошибки E и добавление к общей ошибке
                E_m = 0.5 * np.sum((d_m - y_j)**2)  # MSE (Mean Squared Error) для бинарной классификации
                E_total += E_m

                # Вычисление коррекции синаптических весов
                dw = self.learning_rate * (d_m - y_j) * y_j * (1 - y_j) * X_m[:, np.newaxis] # Добавил np.newaxis для корректной размерности
                db = self.learning_rate * (d_m - y_j) * y_j * (1 - y_j)

                # Накопление изменений весов и смещений
                delta_w += dw
                delta_b += db

            # Коррекция синаптических весов и смещений (после обработки всех векторов)
            self.weights += delta_w / n_samples #  Делим на количество сэмплов, чтобы получить среднее изменение
            self.bias += delta_b / n_samples # Делим на количество сэмплов, чтобы получить среднее изменение

            print(f"Эпоха {epoch+1}, общая ошибка: {E_total / n_samples}")

def load_mnist(digits=(0, 1), n_samples=None):
    try:
        mnist = fetch_openml('mnist_784', version=1, as_frame=False)
        X, y = mnist["data"], mnist["target"]

        X_filtered, y_filtered = [], []
        for i in range(len(y)):
            if y[i] in [str(digits[0]), str(digits[1])]:
                X_filtered.append(X[i])
                y_filtered.append(1 if y[i] == str(digits[1]) else 0)

        X = np.array(X_filtered)
        y = np.array(y_filtered)


        # Ограничение количества образцов (если указано)
        if n_samples is not None:
            X = X[:n_samples]
            y = y[:n_samples]

        # Нормализация данных (масштабирование к диапазону [0, 1])
        X = X / 255.0

        return X, y
    except Exception as e:
        print(f"Ошибка при загрузке MNIST: {e}")
        return None, None



if __name__ == '__main__':
    # 1. Загрузка датасета MNIST
    digits_to_use = (0, 1)
    X, y = load_mnist(digits=digits_to_use, n_samples=2000)

    if X is not None and y is not None:
        # 2. Разделение на обучающую и тестовую выборки
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52)

        # 3. Создание и обучение персептрона
        n_inputs = X_train.shape[1]
        perceptron = Perceptron(n_inputs=n_inputs, learning_rate=0.1, n_iters=20, random_state=52)  # Установите random_state
        perceptron.train(X_train, y_train)

        # 4. Предсказание на тестовой выборке
        y_pred = perceptron.predict(X_test)

        # 5. Оценка точности
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Точность: {accuracy}")


Эпоха 1, общая ошибка: 0.12917019662054824
Эпоха 2, общая ошибка: 0.10954412439758564
Эпоха 3, общая ошибка: 0.09416507523632302
Эпоха 4, общая ошибка: 0.08212288437889942
Эпоха 5, общая ошибка: 0.07250520271106987
Эпоха 6, общая ошибка: 0.06467665412218321
Эпоха 7, общая ошибка: 0.058212576208401305
Эпоха 8, общая ошибка: 0.05281509738598242
Эпоха 9, общая ошибка: 0.04826454330628913
Эпоха 10, общая ошибка: 0.04439376184571064
Эпоха 11, общая ошибка: 0.04107338743377966
Эпоха 12, общая ошибка: 0.038202296740886064
Эпоха 13, общая ошибка: 0.035700859296323634
Эпоха 14, общая ошибка: 0.03350594543295858
Эпоха 15, общая ошибка: 0.03156716485592122
Эпоха 16, общая ошибка: 0.029844012322924208
Эпоха 17, общая ошибка: 0.02830369423765965
Эпоха 18, общая ошибка: 0.02691946871654581
Эпоха 19, общая ошибка: 0.025669373135823067
Эпоха 20, общая ошибка: 0.02453524426385109
Точность: 0.98
